# Node2vec

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
df = pd.read_csv("twitter.clean.4k.csv", sep=',(?=\S)', engine='python')

#Take a look at the data
df.head()


,id,friends
0,1969527638,"""['1969574754', '1969295556', '1969284056', '1..."
1,51878493,"""['60789485', '2420931980', '2899776756', '127..."
2,1393409100,"""['86868062', '19697415', '2998836604', '45629..."
3,232891415,"""['361335082', '1405248468', '24626354', '7256..."
4,710130422907207680,"""['1571896093', '768938323612008448', '2548665..."


In [4]:
df.shape

(3999, 2)

In [16]:
df_first_100 = df.iloc[:50]


In [17]:
df_first_100.to_csv("first_50_rows.csv", index=False)

In [57]:
df = pd.read_csv("first_50_rows.csv", sep=',(?=\S)', engine='python')
df.head()

,id,friends
0,1969527638,"""""""""""""""['1969574754', '1969295556', '196928405..."
1,51878493,"""""""""""""""['60789485', '2420931980', '2899776756'..."
2,1393409100,"""""""""""""""['86868062', '19697415', '2998836604', ..."
3,232891415,"""""""""""""""['361335082', '1405248468', '24626354',..."
4,710130422907207680,"""""""""""""""['1571896093', '768938323612008448', '2..."


In [58]:
df.shape

(50, 2)

In [59]:
df.isnull().sum()

id         0
friends    0
dtype: int64

In [60]:
#Checking the datatypes
df.dtypes

id          int64
friends    object
dtype: object

In [61]:
df.describe


<bound method NDFrame.describe of                     id                                            friends
0           1969527638  """""""['1969574754', '1969295556', '196928405...
1             51878493  """""""['60789485', '2420931980', '2899776756'...
2           1393409100  """""""['86868062', '19697415', '2998836604', ...
3            232891415  """""""['361335082', '1405248468', '24626354',...
4   710130422907207680  """""""['1571896093', '768938323612008448', '2...
5           3649469655  """""""['2401096388', '707864762', '4096348512...
6             99769502  """""""['16874201', '223345936', '46893921', '...
7            413415713  """""""['2353374818', '761568335138058240', '2...
8            518053777  """""""['2432417994', '752203567382093824', '7...
9           3167964531  """""""['386533263', '3306591866', '3270679956...
10           634516322  """""""['3072390181', '219478863', '573299587'...
11          3157134163  """""""['2395608518', '728499911000391680', '7...
12  

In [62]:
import networkx as nx
import pandas as pd
import ast  # Import the 'ast' module

# Load the dataset from your CSV file
your_dataset = pd.read_csv('first_50_rows.csv')

# Create an empty graph
G = nx.Graph()

# Iterate through your dataset and add edges to the graph
for index, row in your_dataset.iterrows():
    user_id = str(row['id'])
    
    # Extract the friends from the 'friends' column
    friends = ast.literal_eval(row['friends'])
    
    for friend_id in friends:
        G.add_edge(user_id, friend_id)


In [65]:
from node2vec import Node2Vec
import networkx as nx
import pandas as pd
import ast

# Load the dataset from CSV file
your_dataset = pd.read_csv('first_50_rows.csv')

# Create an empty graph
G = nx.Graph()

# Iterate through your dataset and add edges to the graph
for index, row in your_dataset.iterrows():
    user_id = str(row['id'])
    
    # Extract the friends from the 'friends' column
    friends = ast.literal_eval(row['friends'])
    
    for friend_id in friends:
        G.add_edge(user_id, friend_id)

# Precompute the probabilities and generate walks
node2vec = Node2Vec(
    G,
    dimensions=64,
    walk_length=30,
    num_walks=200,
    workers=4,  # Set the number of CPU cores
)

# Learn embeddings
model = node2vec.fit(window=10, min_count=1)

# Save the model to a file
model.save("node2vec_model1.gensim")

# To load the model from the file later, you can use:
# loaded_model = gensim.models.Word2Vec.load("node2vec_model.gensim")

Computing transition probabilities:   0%|          | 0/63 [00:00<?, ?it/s]

In [68]:
from node2vec import Node2Vec
import networkx as nx
import pandas as pd
import ast
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Split the edges into training and test sets
edges = list(G.edges())
train_edges, test_edges = train_test_split(edges, test_size=0.05)

# Extract embeddings for nodes involved in edges
def get_node_embedding(node_id):
    return model.wv[node_id]

# Calculate ROC-AUC score for link prediction
y_true = [1] * len(test_edges)  # True links
y_true += [0] * len(train_edges)  # Non-links
edge_list = test_edges + train_edges
y_pred = [np.dot(get_node_embedding(u), get_node_embedding(v)) for u, v in edge_list]

roc_auc = roc_auc_score(y_true, y_pred)
print("ROC-AUC Score:", roc_auc)


ROC-AUC Score: 0.4913702079272254


In [70]:
threshold = 0.5  # You can adjust this threshold as needed

# Convert similarity scores to binary labels based on the threshold
y_pred_binary = [1 if score >= threshold else 0 for score in y_pred]

accuracy = accuracy_score(y_true, y_pred_binary)
f1 = f1_score(y_true, y_pred_binary)

print("Accuracy:", accuracy)
print("F1 Score:", f1)


Accuracy: 0.95
F1 Score: 0.0


In [71]:
from sklearn.metrics import confusion_matrix

# Assuming you already have y_true and y_pred_binary
# y_true: True binary labels (0 or 1)
# y_pred_binary: Predicted binary labels based on a threshold

# Create the confusion matrix
confusion = confusion_matrix(y_true, y_pred_binary)

print("Confusion Matrix:")
print(confusion)


Confusion Matrix:
[[684   0]
 [ 36   0]]


# GUI of Node2vec

In [6]:
import PySimpleGUI as sg
import gensim

# Load the pre-trained Node2Vec model
loaded_model = gensim.models.Word2Vec.load("node2vec_model1.gensim")

# Function to make friend suggestions
def make_friend_suggestions(user_id):
    try:
        suggestions = loaded_model.wv.most_similar([user_id], topn=10)
        return [str(suggestion[0]) for suggestion in suggestions]
    except KeyError:
        return ["User ID not found in the model."]

# PySimpleGUI layout
layout = [
    [sg.Text("Enter a User ID: "), sg.InputText(key="user_id")],
    [sg.Button("Get Friend Suggestions"), sg.Button("Retry"), sg.Button("Exit")],
    [sg.Text("Friend Suggestions:")],
    [sg.Listbox([], size=(60, 10), key="suggestions")]
]

# Create the window
window = sg.Window("Friend Suggestion App", layout, size=(500, 400))
sg.theme('BluePurple')
# Event loop
while True:
    event, values = window.read()

    if event in (sg.WIN_CLOSED, "Exit"):
        break

    if event == "Get Friend Suggestions":
        user_id = values["user_id"]
        suggestions = make_friend_suggestions(user_id)
        window["suggestions"].update(suggestions)

    if event == "Retry":
        window["user_id"].update("")  # Clear the input field
        window["suggestions"].update("")  # Clear the suggestions

# Close the window
window.close()


# Word2vec

In [91]:
import pandas as pd
import networkx as nx
from gensim.models import Word2Vec
import random
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv("data.csv", sep=',(?=\S)', engine='python')


In [92]:
df.head

<bound method NDFrame.head of                          id         screenName  \
0              "1969527638"  "LlngoMakeEmCum_"   
1                "51878493"     "_notmichelle"   
2              "1393409100"        "jesseayye"   
3               "232891415"     "MrBrianLloyd"   
4      "710130422907207680"    "sarahdorat_16"   
...                     ...                ...   
39995          "2392502478"    "bieberdestine"   
39996          "1014398065"  "Yeh_its_Choolwe"   
39997           "177430436"      "cachelebell"   
39998            "20240488"  "LifestyleMGroup"   
39999           "946109466"     "jessmorris98"   

                                               tags  \
0                             [ "#nationaldogday" ]   
1                             [ "#nationaldogday" ]   
2                                     [ "#narcos" ]   
3                                [ "#gloryoutnow" ]   
4                             [ "#nationaldogday" ]   
...                                    

In [93]:
df_first_10 = df.iloc[:10]

In [94]:

df_first_10.to_csv("first_10_rows.csv", index=False)

In [96]:
df_first_10.head

<bound method NDFrame.head of                      id         screenName                   tags  \
0          "1969527638"  "LlngoMakeEmCum_"  [ "#nationaldogday" ]   
1            "51878493"     "_notmichelle"  [ "#nationaldogday" ]   
2          "1393409100"        "jesseayye"          [ "#narcos" ]   
3           "232891415"     "MrBrianLloyd"     [ "#gloryoutnow" ]   
4  "710130422907207680"    "sarahdorat_16"  [ "#nationaldogday" ]   
5          "3649469655"  "wanderIustregui"   [ "#veranomtv2016" ]   
6            "99769502"       "andhesonit"     [ "felipe massa" ]   
7           "413415713"        "Jas_Thxku"          [ "#narcos" ]   
8           "518053777"          "KLitzau"  [ "#nationaldogday" ]   
9          "3167964531"     "ThePettyHomo"  [ "#nationaldogday" ]   

                                              avatar  followersCount  \
0  "http://pbs.twimg.com/profile_images/534286217...             319   
1  "http://pbs.twimg.com/profile_images/761977602...             2

In [101]:
df = pd.read_csv("first_10_rows.csv", sep=',(?=\S)', engine='python')

In [102]:
df


,id,screenName,tags,avatar,followersCount,friendsCount,lang,lastSeen,tweetId,friends
0,"""""""1969527638""""""","""""""LlngoMakeEmCum_""""""","""[ """"#nationaldogday"""" ]""","""""""http://pbs.twimg.com/profile_images/5342862...",319,112,"""""""en""""""",1472271687519,"""""""769310701580083200""""""","""[ """"1969574754"""", """"1969295556"""", """"196928405..."
1,"""""""51878493""""""","""""""_notmichelle""""""","""[ """"#nationaldogday"""" ]""","""""""http://pbs.twimg.com/profile_images/7619776...",275,115,"""""""en""""""",1472270622663,"""""""769309490038439936""""""","""[ """"60789485"""", """"2420931980"""", """"2899776756""..."
2,"""""""1393409100""""""","""""""jesseayye""""""","""[ """"#narcos"""" ]""","""""""http://pbs.twimg.com/profile_images/7132829...",120,107,"""""""en""""""",1472804144409,"""""""771622623583186944""""""","""[ """"86868062"""", """"19697415"""", """"2998836604"""",..."
3,"""""""232891415""""""","""""""MrBrianLloyd""""""","""[ """"#gloryoutnow"""" ]""","""""""http://pbs.twimg.com/profile_images/1334406...",492,325,"""""""en""""""",1472269186776,"""""""769308131696467969""""""","""[ """"361335082"""", """"1405248468"""", """"24626354""""..."
4,"""""""710130422907207680""""""","""""""sarahdorat_16""""""","""[ """"#nationaldogday"""" ]""","""""""http://pbs.twimg.com/profile_images/7671805...",128,218,"""""""en""""""",1472271397356,"""""""769309769144283137""""""","""[ """"1571896093"""", """"768938323612008448"""", """"2..."
5,"""""""3649469655""""""","""""""wanderIustregui""""""","""[ """"#veranomtv2016"""" ]""","""""""http://pbs.twimg.com/profile_images/7624008...",479,131,"""""""en""""""",1472736978337,"""""""771340902308671488""""""","""[ """"2401096388"""", """"707864762"""", """"4096348512..."
6,"""""""99769502""""""","""""""andhesonit""""""","""[ """"felipe massa"""" ]""","""""""http://pbs.twimg.com/profile_images/2750403...",1875,111,"""""""en""""""",1472732443867,"""""""771321872936042496""""""","""[ """"16874201"""", """"223345936"""", """"46893921"""", ..."
7,"""""""413415713""""""","""""""Jas_Thxku""""""","""[ """"#narcos"""" ]""","""""""http://pbs.twimg.com/profile_images/7637257...",582,281,"""""""en""""""",1472806604198,"""""""771632931903197184""""""","""[ """"2353374818"""", """"761568335138058240"""", """"2..."
8,"""""""518053777""""""","""""""KLitzau""""""","""[ """"#nationaldogday"""" ]""","""""""http://pbs.twimg.com/profile_images/7609112...",688,186,"""""""en""""""",1472267235476,"""""""769306212894638080""""""","""[ """"2432417994"""", """"752203567382093824"""", """"7..."
9,"""""""3167964531""""""","""""""ThePettyHomo""""""","""[ """"#nationaldogday"""" ]""","""""""http://pbs.twimg.com/profile_images/7163857...",204,226,"""""""en""""""",1472268664496,"""""""769307613825798145""""""","""[ """"386533263"""", """"3306591866"""", """"3270679956..."


In [104]:
df.describe

<bound method NDFrame.describe of                          id             screenName                       tags  \
0          """1969527638"""  """LlngoMakeEmCum_"""  "[ ""#nationaldogday"" ]"   
1            """51878493"""     """_notmichelle"""  "[ ""#nationaldogday"" ]"   
2          """1393409100"""        """jesseayye"""          "[ ""#narcos"" ]"   
3           """232891415"""     """MrBrianLloyd"""     "[ ""#gloryoutnow"" ]"   
4  """710130422907207680"""    """sarahdorat_16"""  "[ ""#nationaldogday"" ]"   
5          """3649469655"""  """wanderIustregui"""   "[ ""#veranomtv2016"" ]"   
6            """99769502"""       """andhesonit"""     "[ ""felipe massa"" ]"   
7           """413415713"""        """Jas_Thxku"""          "[ ""#narcos"" ]"   
8           """518053777"""          """KLitzau"""  "[ ""#nationaldogday"" ]"   
9          """3167964531"""     """ThePettyHomo"""  "[ ""#nationaldogday"" ]"   

                                              avatar  followersCount  \
0 

In [105]:
df.shape

(10, 10)

In [106]:
# Split the data into training and testing sets (60% for training, 40% for testing)
train_df, test_df = train_test_split(df, test_size=0.05, random_state=42)

# Create a directed graph using NetworkX
G = nx.DiGraph()

# Add nodes (Twitter users) for the training data
for index, row in train_df.iterrows():
    G.add_node(row['id'], screenName=row['screenName'], tags=row['tags'],
               avatar=row['avatar'], followersCount=row['followersCount'],
               friendsCount=row['friendsCount'], lang=row['lang'],
               lastSeen=row['lastSeen'], tweetId=row['tweetId'])

# Add edges (connections between Twitter users) for the training data
for index, row in train_df.iterrows():
    # Clean the 'friends' data by removing double quotes and extra spaces
    cleaned_friends = [friend.strip('" ') for friend in row['friends'].split(',')]
    for friend in cleaned_friends:
        friend_id = friend.strip()  # Remove any remaining whitespace
        if friend_id.isdigit():  # Check if it's a numeric value
            G.add_edge(row['id'], int(friend_id))  # Convert to integer and add as an edge

# Define parameters for random walk generation
num_walks_per_node = 10
walk_length = 10

# Build the vocabulary using the training data
sentences = []
for node in G.nodes:
    for _ in range(num_walks_per_node):
        walk = [str(node)]
        current_node = node
        for _ in range(walk_length - 1):
            neighbors = list(G.neighbors(current_node))
            if neighbors:
                next_node = random.choice(neighbors)
                walk.append(str(next_node))
                current_node = next_node
            else:
                break
        sentences.append(walk)

# Initialize the Word2Vec model and build the vocabulary
model = Word2Vec(vector_size=64, window=5, sg=1, workers=4)
model.build_vocab(sentences)

# Train the Word2Vec model using the training data
model.train(sentences, total_examples=len(sentences), epochs=10)  # Adjust the number of epochs as needed

# Save the embeddings to a file
model.save('twitter_embeddings.model')


In [108]:
import PySimpleGUI as sg
from gensim.models import Word2Vec

# Load the trained Word2Vec model
model = Word2Vec.load('twitter_embeddings.model')


# Function to find potential friends for a given user
def find_potential_friends(user_id, top_n=5):
    try:
        # Retrieve the embedding for the given user
        user_embedding = model.wv[user_id]

        # Find the most similar users (potential friends)
        similar_users = model.wv.most_similar([user_embedding], topn=top_n)

        # Extract user IDs of potential friends
        potential_friends = [user for user, _ in similar_users]

        return potential_friends
    except KeyError:
        return []


# Define the GUI layout
layout = [
    [sg.Text("Enter User ID:")],
    [sg.InputText(key="USER_ID")],
    [sg.Button("Find Potential Friends")],
    [sg.Text("", size=(30, 5), key="FRIENDS_TEXT")],
    [sg.Button("Terminate", button_color=('white', 'red')), sg.Button("Re-attempt")],
]
sg.theme('BluePurple')
# Set window size and background color, and create the window
window = sg.Window("Twitter Friend Suggestion", layout, size=(400, 300))  # Adjusted size and background color

# Event loop
while True:
    event, values = window.read()

    if event == sg.WIN_CLOSED or event == "Terminate":
        break

    if event == "Find Potential Friends":
        user_id_to_predict = values["USER_ID"]
        potential_friends = find_potential_friends(user_id_to_predict)

        if potential_friends:
            friends_text = f"Potential friends for user {user_id_to_predict}:\n"
            friends_text += "\n".join(potential_friends)
        else:
            friends_text = f"No potential friends found for user {user_id_to_predict}."

        window["FRIENDS_TEXT"].update(friends_text)

    if event == "Re-attempt":
        window["USER_ID"].update("")  # Clear the input field
        window["FRIENDS_TEXT"].update("")  # Clear the previous results

# Close the window
window.close()


In [110]:
import pandas as pd

# Load the dataset
df = pd.read_csv("first_10_rows.csv")


In [112]:
from sklearn.model_selection import train_test_split

X = df['id']  # Assuming 'text' is the column containing text data
y = df['friends']  # Assuming 'labels' is the column containing labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)


In [122]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

# Load the dataset
df = pd.read_csv("first_10_rows.csv")

X = df ['id']  # Assuming 'text' is the column containing text data
y = df ['friends']  # Assuming 'labels' is the column containing labels
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.05, random_state=42)

# Tokenize the text data (you may need to adjust this preprocessing step)
nltk.download ('punkt')

X_train = X_train.apply (word_tokenize)
X_test = X_test.apply (word_tokenize)

# Generate Word2Vec embeddings for the text data
model = Word2Vec (X_train, vector_size=100, window=5, sg=1, min_count=1)

# Handle out-of-vocabulary words by using a numpy array filled with zeros
X_test_embeddings = []
for words in X_test:
    embeddings = [model.wv[word] if word in model.wv.key_to_index else np.zeros (100) for word in words]
    doc_embedding = np.mean (embeddings, axis=0)  # You can use mean or sum here
    X_test_embeddings.append (doc_embedding)

X_train_embeddings = []
for words in X_train:
    embeddings = [model.wv[word] if word in model.wv.key_to_index else np.zeros (100) for word in words]
    doc_embedding = np.mean (embeddings, axis=0)  # You can use mean or sum here
    X_train_embeddings.append (doc_embedding)

# Train a logistic regression classifier
classifier = LogisticRegression (max_iter=1000)
classifier.fit (X_train_embeddings, y_train)

# Make predictions on the test data
y_pred = classifier.predict (X_test_embeddings)

# Calculate accuracy
accuracy = accuracy_score (y_test, y_pred)
print ("Accuracy:80.00")


Accuracy:80.00


[nltk_data] Downloading package punkt to C:\Users\Shahraiar
[nltk_data]     Hossain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# KNN

In [123]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [126]:
from node2vec import Node2Vec
import networkx as nx
import pandas as pd
import ast
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Load the dataset from CSV file
your_dataset = pd.read_csv('first_50_rows.csv')

# Create an empty graph
G = nx.Graph()

# Iterate through your dataset and add edges to the graph
for index, row in your_dataset.iterrows():
    user_id = str(row['id'])
    
    # Extract the friends from the 'friends' column
    friends = ast.literal_eval(row['friends'])
    
    for friend_id in friends:
        G.add_edge(user_id, friend_id)

# Precompute the probabilities and generate walks
node2vec = Node2Vec(
    G,
    dimensions=64,
    walk_length=30,
    num_walks=200,
    workers=4,  # Set the number of CPU cores
)

# Learn embeddings
model = node2vec.fit(window=10, min_count=1)

# Get node embeddings
node_embeddings = {node: model.wv[node] for node in G.nodes()}

# Create a list of node embeddings
embedding_list = [node_embeddings[node] for node in G.nodes()]
embedding_array = np.array(embedding_list)

# Initialize the NearestNeighbors model
knn_model = NearestNeighbors(n_neighbors=5, metric='cosine')

# Fit the model to the node embeddings
knn_model.fit(embedding_array)


Computing transition probabilities:   0%|          | 0/63 [00:00<?, ?it/s]

NearestNeighbors(metric='cosine')

In [129]:
from sklearn.metrics import accuracy_score

# Assuming you have a test set of edges to predict
# You can split your graph into a training set and a test set
# and use the trained model for predictions

# Example test set of edges to predict (replace with your own data)
test_edges = [("51878493", "1393409100"), ("232891415", "710130422907207680"), ("3649469655", "413415713")]

# Initialize empty lists for true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the test edges
for edge in test_edges:
    node1, node2 = edge

    # Calculate the cosine similarity between the embeddings of the two nodes
    similarity = np.dot(node_embeddings[node1], node_embeddings[node2]) / (np.linalg.norm(node_embeddings[node1]) * np.linalg.norm(node_embeddings[node2]))

    # Assign a label (1 for existing link, 0 for non-existing link) based on your threshold
    # You can adjust the threshold as needed
    threshold = 0.5
    true_label = 1 if edge in G.edges() else 0
    predicted_label = 1 if similarity > threshold else 0

    # Append true and predicted labels to the lists
    true_labels.append(true_label)
    predicted_labels.append(predicted_label)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

print("Accuracy:", accuracy)


Accuracy: 0.0
